1. 1 слой 26 нейронов (сначала передать туда матрицу декодера, потом инициализировать рандомно) 
2. 2 слоя по 31 нейрона
3. 

1. бинарный вход перевести в вещественный в [0,1]


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
# !pip install -q keras
# !pip install -q pandas
# !pip install -q numpy

In [0]:
import os 
import sys
import pandas as pd
import numpy as np

from sklearn import model_selection
from __future__ import absolute_import

os.chdir("/content/drive/Hamming")
sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [5]:
!ls
import tensorflow as tf
tf.test.gpu_device_name()

best_model.hd5	functions.py	     README.md
dataset_files	hamming-codes.ipynb  Untitled0.ipynb
drive		model_lib.py	     visualisation.ipynb


'/device:GPU:0'

In [0]:
# !git pull

### hamming_small.txt - маленький датасет
### hamming.txt - большой датасет

In [0]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [0]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [0]:
def load_data():
    return pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

In [0]:
# def balance_classes():
#     '''
#     Датасет копируется в no_err, 
#     значение маски ошибки заменяется на нулевую маску, 
#     значение "испорченного" кодового слова заменяется на кодовое,
#     no_err добавляется к датасету в конце.
#     '''
    
#     no_err = data.copy()  # deep=True)
#     no_err['id_error'] = -1
#     no_err['bin_error'] = '0' * 31
#     no_err['defective_codeword'] = no_err['codeword']
#     return data.append(no_err, ignore_index=True)  # ignore_index для сквозной нумеракции итогового датасета


In [0]:
def make_features():
#     data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['codeword'][0])):
        data['cod_' + str(j)] = data['codeword'][:].apply(lambda x: int(x[j]))

    for j in range(len(data['defective_codeword'][0])):
        data['def_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))

#     for j in range(len(data['bin_error'][0])):
#         data['mask_' + str(j)] = data['bin_error'][:].apply(lambda x: int(x[j]))
        
    for j in range(len(data['plainword'][0])):
        data['pln_' + str(j)] = data['plainword'][:].apply(lambda x: int(x[j]))
#     return data

In [0]:
def split_data(test_size): 
  train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'def_0':'def_30'], 
      data.loc[:, 'cod_0':'cod_30'], # 'mask_0':'pln_25' 
      test_size = test_size) 
  return np.array(train_data), np.array(test_data), np.array(train_labels), np.array(test_labels)

In [32]:
data = load_data()
print(data.shape)

(2097152, 5)


In [0]:
# data = balance_classes()
# print(data.shape)

In [34]:
%%time
make_features()
print(data.shape)

(2097152, 93)
CPU times: user 1min 47s, sys: 1.63 s, total: 1min 49s
Wall time: 1min 47s


In [35]:
data.head()

,plainword,codeword,id_error,bin_error,defective_codeword,cod_0,cod_1,cod_2,cod_3,cod_4,...,pln_16,pln_17,pln_18,pln_19,pln_20,pln_21,pln_22,pln_23,pln_24,pln_25
0,00110111111011010110111000,0011100110111111011010110111000,-1,0000000000000000000000000000000,0011100110111111011010110111000,0,0,1,1,1,...,0,1,1,0,1,1,1,0,0,0
1,00110111111011010110111000,0011100110111111011010110111000,0,1000000000000000000000000000000,1011100110111111011010110111000,0,0,1,1,1,...,0,1,1,0,1,1,1,0,0,0
2,00110111111011010110111000,0011100110111111011010110111000,1,0100000000000000000000000000000,0111100110111111011010110111000,0,0,1,1,1,...,0,1,1,0,1,1,1,0,0,0
3,00110111111011010110111000,0011100110111111011010110111000,2,0010000000000000000000000000000,0001100110111111011010110111000,0,0,1,1,1,...,0,1,1,0,1,1,1,0,0,0
4,00110111111011010110111000,0011100110111111011010110111000,3,0001000000000000000000000000000,0010100110111111011010110111000,0,0,1,1,1,...,0,1,1,0,1,1,1,0,0,0


In [0]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [0]:
# print(train_data)
# decoder = []
# with open('./dataset_files/h.txt') as f:
#     decoder = pd.read_csv(f, header=None, names=['h'], dtype=str) #, sep='\n')
# for j in range(len(decoder['h'][0])):
#     decoder['bin_' + str(j)] = decoder['h'][:].apply(lambda x: int(x[j]))
# decoder = decoder.drop(['h'], axis=1)
# decoder_np = decoder.as_matrix()
# print(decoder_np[:6])

In [0]:
# model = Sequential()
# model.add(Dense(31, input_shape=(31,), 
# #                 weights=decoder_np, 
#                 activation='relu', name='d1'))
# # model.add(Dense(100, activation='relu', name='d2'))
# # model.add(Dense(31, activation='softmax', name='d3'))
# model.compile(optimizer='rmsprop',
#               loss='categorical_crossentropy', # энтропия Шелдона / бинарная
#               metrics=['accuracy', 'mse'])

MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)

In [0]:
# train_labels_categor = to_categorical(train_labels, num_classes=31)

model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

LSTM;

матринчая нейронная сеть, каждый слой - логические ячейки xor...
метрика хэмминга

подавать и правильные и неправильные


поставить керас
from keras... import
inp = Input(shape=(31,))
x = Dense(128, activation='elu')(inp)
x = Dense(128, activation='relu')(x)

test = Dense(.....)(x)

m = Dense(26, activation='пороговая функция типа step которой уже нет 'sigmoid')(x)

test = Dense(...)(test)
test= Dense(31....)(test)
m = Concatinate()([test,m])
          
model = Model(inp, m, 'm')
model.compile() , summary, fit

метки = бинарный код ошибки + информационное слово

лосс - бинарная кросс энтропия
оптимизатор - Адам
стартовая точноесть - 1е-5

In [0]:
# from keras.layers import Conv1D, Reshape, Flatten

# inp = Input(shape=(31,))
# x = Dense(128, activation='relu',  name='d1')(inp)
# x = Dense(128, activation='relu',  name='d2')(x)

# t = Dense(128, activation='elu', name='t1')(x)
# t = Dense(128, activation='relu', name='t2')(t)
# t = Dense(31, activation='softmax', name='t3')(t)

# m = Dense(26, activation='sigmoid',  name='d3')(x)
# m = Concatenate()([t, m])

# model = Model(inp, m, 'm')
# model.compile(loss='binary_crossentropy', optimizer='Adam')
# model.summary()

In [16]:
from keras.layers import Conv1D, Reshape, Flatten, MaxPooling1D, UpSampling1D
from keras.layers.recurrent import LSTM
from keras import models
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate, CuDNNLSTM
# from keras.utils import to_categorical
# from keras.callbacks import ModelCheckpoint

inp = Input(shape=(31,))
x = Dense(256, activation='tanh')(inp)
x = Dense(512, activation='softsign')(x)
x = Dense(1024, activation='sigmoid')(x)
x = Dense(512, activation='softsign')(x)
x = Dense(256, activation='tanh')(x)
# x = Dense(128, activation='softmax')(x)
# x = Dense(1024, activation='relu')(x)
# x = Dense(1054, activation='relu')(x)

m = Dense(31, activation='sigmoid',  name='d3')(x)

model = Model(inp, m, 'm')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 31)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8192      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
d3 (Dense)                   (None, 31)                7967      
Total para

Using TensorFlow backend.


In [17]:
model.fit(x=train_data, y=train_labels, validation_split=0.15,  shuffle=True, epochs=15, batch_size=32)#, callbacks=[MCP])tu

Train on 38993 samples, validate on 6882 samples
Epoch 1/15
38993/38993 [==============================] - 12s 299us/step - loss: 0.1930 - acc: 0.9366 - val_loss: 0.1246 - val_acc: 0.9675
Epoch 2/15
38993/38993 [==============================] - 11s 291us/step - loss: 0.0955 - acc: 0.9712 - val_loss: 0.0885 - val_acc: 0.9722
Epoch 3/15
 5664/38993 [===>..........................] - ETA: 9s - loss: 0.0672 - acc: 0.9765

38993/38993 [==============================] - 11s 290us/step - loss: 0.0606 - acc: 0.9780 - val_loss: 0.0662 - val_acc: 0.9770
Epoch 4/15
38993/38993 [==============================] - 11s 290us/step - loss: 0.0361 - acc: 0.9858 - val_loss: 0.0440 - val_acc: 0.9824
Epoch 5/15
20544/38993 [==============>...............] - ETA: 5s - loss: 0.0216 - acc: 0.9918

38993/38993 [==============================] - 11s 290us/step - loss: 0.0206 - acc: 0.9921 - val_loss: 0.0328 - val_acc: 0.9864
Epoch 6/15
38993/38993 [==============================] - 11s 288us/step - loss: 0.0111 - acc: 0.9962 - val_loss: 0.0257 - val_acc: 0.9894
Epoch 7/15
24608/38993 [=================>............] - ETA: 3s - loss: 0.0057 - acc: 0.9985

38993/38993 [==============================] - 11s 289us/step - loss: 0.0057 - acc: 0.9984 - val_loss: 0.0218 - val_acc: 0.9913
Epoch 8/15
38993/38993 [==============================] - 11s 292us/step - loss: 0.0029 - acc: 0.9994 - val_loss: 0.0179 - val_acc: 0.9929
Epoch 9/15
24672/38993 [=================>............] - ETA: 4s - loss: 0.0016 - acc: 0.9998

38993/38993 [==============================] - 11s 294us/step - loss: 0.0016 - acc: 0.9998 - val_loss: 0.0166 - val_acc: 0.9936
Epoch 10/15
38993/38993 [==============================] - 11s 291us/step - loss: 8.4185e-04 - acc: 0.9999 - val_loss: 0.0147 - val_acc: 0.9944
Epoch 11/15
23552/38993 [=================>............] - ETA: 4s - loss: 4.8014e-04 - acc: 1.0000

38993/38993 [==============================] - 11s 293us/step - loss: 5.0018e-04 - acc: 1.0000 - val_loss: 0.0144 - val_acc: 0.9948
Epoch 12/15
38993/38993 [==============================] - 11s 288us/step - loss: 2.9778e-04 - acc: 1.0000 - val_loss: 0.0121 - val_acc: 0.9956
Epoch 13/15
22976/38993 [================>.............] - ETA: 4s - loss: 1.8142e-04 - acc: 1.0000

38993/38993 [==============================] - 11s 292us/step - loss: 2.0248e-04 - acc: 1.0000 - val_loss: 0.0113 - val_acc: 0.9958
Epoch 14/15
38993/38993 [==============================] - 11s 293us/step - loss: 1.3844e-04 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 0.9965
Epoch 15/15
22464/38993 [================>.............] - ETA: 4s - loss: 6.8604e-05 - acc: 1.0000

38993/38993 [==============================] - 11s 290us/step - loss: 7.3238e-05 - acc: 1.0000 - val_loss: 0.0087 - val_acc: 0.9970


### Проверим, сколько слов исправлено точно

In [0]:
y_pred = model.predict(test_data)

In [38]:
print(test_data.shape[0])
test_data.shape[0] / 32

629146


19660.8125

In [39]:
print(y_pred.shape)
# print(test_labels[0], test_data[0])

(629146, 31)


In [0]:
def probs_to_labels(predicted_probs):
    return [1 if x > 0.5 else 0 for x in predicted_probs]
def count_errors(y, y_pred):
  count = 0
  for i in range (0,31):
    labelBit = y[i]
    resultBit = y_pred[i]
    if labelBit != resultBit:
      count += 1
  return count


In [40]:
errorStats = {'0': 0}
for i in range(0, y_pred.shape[0]):
  resultArray = probs_to_labels(y_pred[i])
  errorNum = count_errors(test_labels[i], resultArray)
  if errorStats.get(str(errorNum)) == None:
    errorStats[str(errorNum)] = 0
  errorStats[str(errorNum)] += 1
print(errorStats)

{'0': 19233, '1': 470064, '2': 122632, '3': 15873, '4': 1276, '5': 64, '6': 4}


In [0]:
def binary_accuracy(y, y_pred):
    return sum(1 if np.array_equal(a, probs_to_labels(b)) else 0 for (a,b) in zip(y, y_pred)) / y.shape[0]

In [26]:
accuracy = binary_accuracy(test_labels, y_pred)
print(accuracy)

0.9064645745384263


### optimizers scores:
- Adam - 0.3 accuracy
- SGD - 0.05
- RMSprop - 0.29

In [29]:
print(accuracy)

0.2995600385284179


0. Сделать датасет поменьше
1. Поменять кодирующую матрицу Хэмминга (генерирующую - g.txt) -> Попробовать потренировать
2. Прочитать статью Бернштейна и оттуда перенять архитектуру


1. смотреть насколько маски ошибки отличаются от предикта
2. гиперпараметры (генеративная матрица, количество слоев, что предсказывается)
3. напомнить про ЛСТМ
4. попробовать GRU
5. добавлять 20 ошибо, а не в каждый бит (случайно)
6. генерить не первае 2**16, а рандомные слова